In [39]:
import numpy as np
import piquasso as pq

import itertools
import functools

from typing import List


updates = 10

shots = 5

M = d = 3  # modes

eta = learning_rate = 0.1


def generate_symmetric_matrix(N):
    A = np.random.rand(N, N)

    return A + A.T


Q = np.array(
    [
        [1.16806055, 0.37167793, 0.21938486],
        [0.37167793, 1.94923447, 0.56833681],
        [0.21938486, 0.56833681, 0.89291145],
    ]
)


def calculate_QUBO_explicitely(Q):
    d = len(Q)

    bitstrings = list(map(np.array, list(itertools.product([0, 1], repeat=d))))

    values = []

    for bitstring in bitstrings:
        values.append(bitstring @ Q @ bitstring)

    return min(values), bitstrings[np.argmin(values)]



def map_to_bitstring(sample: np.ndarray, parity: int) -> np.ndarray:
    return list(map(lambda x: (x + parity) % 2, sample))

def get_samples(thetas: List[float]):
    with pq.Program() as program:
        pq.Q(0, 1, 2) | pq.StateVector((1, 1, 1))

        pq.Q(0, 1) | pq.Beamsplitter(thetas[0])
        pq.Q(1, 2) | pq.Beamsplitter(thetas[1])
        pq.Q(0, 2) | pq.Beamsplitter(thetas[2])

        pq.Q() | pq.ParticleNumberMeasurement()

    simulator = pq.PureFockSimulator(d=3)

    return simulator.execute(program, shots).samples


min_energy = 10000

energy_mean = 0

thetas = [np.pi / 3, np.pi / 4, np.pi / 5]

def get_energy(thetas: List[float], mapper) -> float:
    samples = get_samples(thetas)
    bitstrings = list(map(mapper, samples))
    energies = [bitstring @ Q @ bitstring for bitstring in bitstrings]
    return sum(energies) / shots


for parity in (0, 1):
    map_to_bistring_with_parity = functools.partial(map_to_bitstring, parity=parity)
    for _ in range(updates):

        samples = get_samples(thetas)

        bitstrings = list(map(map_to_bistring_with_parity, samples))
        energies = [bitstring @ Q @ bitstring for bitstring in bitstrings]
        energy_mean = sum(energies) / shots

        print("Energies:", energies)

        if min(energies) < min_energy:
            min_energy = min(energies)
            min_bistring = bitstrings[np.argmin(energies)]

        derivatives = []

        for j in range(len(thetas)):
            upshifted_thetas = np.copy(thetas)
            upshifted_thetas[j] += np.pi / 2
            upshifted_energy = get_energy(upshifted_thetas, mapper=map_to_bistring_with_parity)

            downshifted_thetas = np.copy(thetas)
            downshifted_thetas[j] -= np.pi / 2
            downshifted_energy = get_energy(upshifted_thetas, mapper=map_to_bistring_with_parity)

            derivative = (upshifted_energy - downshifted_energy) / 2

            thetas[j] -= eta * derivative

        #print("Thetas:", thetas)

print("Algo:", min_energy, min_bistring)

print("Exact solution:", calculate_QUBO_explicitely(Q))


Energies: [1.94923447, 6.329005669999999, 1.94923447, 0.89291145, 0.89291145]
Energies: [1.16806055, 1.16806055, 1.16806055, 1.16806055, 1.94923447]
Energies: [1.94923447, 0.89291145, 1.94923447, 1.16806055, 0.89291145]
Energies: [1.94923447, 0.89291145, 0.89291145, 0.89291145, 1.16806055]
Energies: [0.89291145, 6.329005669999999, 1.16806055, 0.89291145, 1.94923447]
Energies: [0.89291145, 1.94923447, 1.94923447, 1.16806055, 1.94923447]
Energies: [1.16806055, 1.94923447, 1.16806055, 0.89291145, 0.89291145]
Energies: [0.89291145, 0.89291145, 1.94923447, 1.16806055, 0.89291145]
Energies: [1.94923447, 0.89291145, 0.89291145, 0.89291145, 1.94923447]
Energies: [1.16806055, 0.89291145, 1.94923447, 1.94923447, 1.94923447]
Energies: [3.97881954, 2.4997417200000003, 3.97881954, 2.4997417200000003, 2.4997417200000003]
Energies: [0.0, 3.97881954, 3.86065088, 3.86065088, 2.4997417200000003]
Energies: [3.86065088, 2.4997417200000003, 3.86065088, 2.4997417200000003, 2.4997417200000003]
Energies: [3.9